This script is meant to replicate the block that reading brain activity from neurons and processing it to provide the BMI with spike binned information.

### Imports

In [1]:
import paho.mqtt.client as paho
import numpy as np
from time import sleep
from datetime import datetime
import json

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import sys

def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

### Parameters

In [3]:
clientID = 'brain_decoder'
broker = 'localhost'
port = 1883
topic = 'brain_activity'

### Script

In [4]:
# Initialize the publisher
client = paho.Client(clientID, False)

In [5]:
# Load the testing samples
# test_trials should be a dict {'box':[np.array with shape NCH, NBINS], 'ball':[], 'ring':[]} 
fake = np.load(f'temp/MRec40_go_X.npy')
test_trials = {
    'ball': fake[0],
    'box': fake[1],
    'ring': fake[2]
              }

In [ ]:
# Start the script, waiting for user input (should be the test to perform)
while True:
    action = input('Which trial to simulate?\n Box, ball, ring or exit:\t')
    action = action.strip().lower()
    if action in ['exit', 'e']:
        break
        
    if action not in ['box', 'ball', 'ring']:
        print('Incorrect input')
        continue
        
    # select the desired trial
    trial = test_trials[action]
    (channels, nbins) = trial.shape
    
    # start the client 
    client.connect(broker, port)
    client.loop_start()
    client.publish(topic, json.dumps({"status": "starting_simulation", "shape": channels}), 2)
    
    try:
        for idx in progressbar(range(nbins), "Simulating: ", 60):
            time_bin = np.squeeze(trial[:, idx])
            # Every 10ms prepare a new json with status and column of the test until finishing the trial 
            msg = json.dumps({
                "name": "Binned brain activity",
                "description": "Recording from all the channels with datetime and simulation index",
                "value": {
                    "measurements": time_bin.tolist(),
                    "datetime": datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
                    "index": idx
                },
                "status": "running_simulation"
            })
            client.publish(topic, msg, 2)
            sleep(1)
            sleep(0.01)
            
        client.publish(topic, json.dumps({"status": "ending_simulation"}), 2)
        client.loop_stop()
        client.disconnect()
    
    except KeyboardInterrupt:
        print('Simulation interrupted')
        client.loop_stop()
        client.disconnect()
        sys.exit(0)
    
    print('\n')

Which trial to simulate?
 Box, ball, ring or exit:	ball
Simulating: [############################################################] 15/15


